## Tutorial: Encode Genetic Perturbagens (Genes/Proteins) with UniPert


In this tutorial, we will introduce `2` ways for obtaining `genetic perturbagen embeddings` using `UniPert`:


1. **Generating from a FASTA File**: By providing a `FASTA file` containing `amino acid sequences`, we can directly generate UniProt embeddings of the entries in the FASTA file. This method is suitable for cases where users already downloaded the sequence information of query perturbagens and is `faster` since it does not require sequence retrieval.

2. **Generating from a Gene Name List**: If you have a list of `gene names`, we can generate the embeddings by querying the corresponding amino acid sequences through the `UniProt API`. This method is useful when you have gene names but not the direct sequence data, but it requires `internet access` and is `slower` due to the need for sequence retrieval.

Please follow the steps and choose the appropriate method based on your needs:

1. [Prepare UniPert model](#prepare-unipert-model)
2. [Generate perturbagen embeddings](#generate-perturbagen-embeddings)
   
   * [From FASTA File](#from-fasta-file)
   * [From Gene Name List](#from-gene-name-list)
  
3. [Save embeddings](#save-embedddings)


## Prepare UniPert model

In [2]:
import sys
sys.path.append('../')
from unipert import UniPert

unipert = UniPert()

💡 CUDA is available. Using CUDA.
💡 Constructing UniPert model...
✅ ESM2 model loaded.
✅ Reference ESM2 embedding file loaded.
✅ ESM2 embedder created.
✅ ECFP4 embedder loaded.
✅ UniPert model constructed.
✅ Pretrained model file loaded.
✅ Reference target graph prepared.
✅ Model loaded and initialized.


## Generate perturbagen embeddings

### From FASTA File

#### Generate UniPert Embeddings for FASTA file

Example FASTA file downloaded from the UniProt website.


In [2]:
out_embs = unipert.enc_gene_ptbgs_from_fasta(
    custom_seq_fasta='../demo_data/UniProt_target_sequence.fasta', 
    save=False
    )

💡 Constructing reference-custom target graph from ../demo_data/UniProt_target_sequence.fasta...
💡 Preparing MMseqs and creating reference database...
[16:18:00 +08:00] [mmseqs] [---I---] [thread 173831] Converting sequences
[16:18:00 +08:00] [mmseqs] [---I---] [thread 173831] 
[16:18:00 +08:00] [mmseqs] [---I---] [thread 173831] Time for merging to ref_h: 0h 0m 0s 204ms
[16:18:00 +08:00] [mmseqs] [---I---] [thread 173831] Time for merging to ref: 0h 0m 0s 14ms
[16:18:00 +08:00] [mmseqs] [---I---] [thread 173831] Database type: Aminoacid
[16:18:00 +08:00] [mmseqs] [---I---] [thread 173831] Time for processing: 0h 0m 0s 743ms
✅ MMseqs reference database created.
💡 Calculating similarity between ../demo_data/UniProt_target_sequence.fasta and reference fasta file...
[16:18:00 +08:00] [mmseqs] [---I---] [thread 173831] Temporary path /fionafyang/lynnymli/scPerturb/mmseqs_storage/workdir/tmp_FbHbU2M5mCCxMA0yodbgbrjyR does not exist or is not a directory. It will be created.
[16:18:00 +08:00]

100%|██████████| 1/1 [00:00<00:00,  1.02it/s]

✅ ESM2 embeddings with 2 querys generated.


In [3]:
out_embs.keys()

dict_keys(['Q9NZQ7'])

### From Gene Name List

#### Generate UniPert embeddings for gene name list

UniPert will retrieve corrosponding canonical amino acid sequences via UniProt website API and generate the embeddings.

In [4]:
gn_list = ['ETV7', 'IFNGR1', 'IRF7', 'PDL1', 'MARCH8', 'IRF1', 'IFNGR2', 'STAT2', 'ATF2', 'CAV1']

In [5]:
out_embs, invalid_inputs = unipert.enc_gene_ptbgs_from_gene_names(gene_names=gn_list)

✅ Referece representations loaded.
✅ 19187 ref targets encoded.
💡 Encoding 10 genetic perturbagens with UniPert...


100%|██████████| 10/10 [00:01<00:00,  6.20it/s]


💡 Constructing reference-custom target graph from ../demo_data/UniProt_target_sequence.fasta...
💡 Calculating similarity between ../demo_data/UniProt_target_sequence.fasta and reference fasta file...
[16:18:51 +08:00] [mmseqs] [---I---] [thread 173831] Temporary path /fionafyang/lynnymli/scPerturb/mmseqs_storage/workdir/tmp_9RnEZ0jiaFz6W6BI7w5T92sbq does not exist or is not a directory. It will be created.
[16:18:51 +08:00] [mmseqs] [---I---] [thread 173831] Created temporary directory /fionafyang/lynnymli/scPerturb/mmseqs_storage/workdir/tmp_9RnEZ0jiaFz6W6BI7w5T92sbq
[16:18:51 +08:00] [mmseqs] [---I---] [thread 173831] Converting sequences
[16:18:51 +08:00] [mmseqs] [---I---] [thread 173831] 
[16:18:51 +08:00] [mmseqs] [---I---] [thread 173831] Time for merging to query_h: 0h 0m 0s 10ms
[16:18:51 +08:00] [mmseqs] [---I---] [thread 173831] Time for merging to query: 0h 0m 0s 10ms
[16:18:51 +08:00] [mmseqs] [---I---] [thread 173831] Database type: Aminoacid
[16:18:51 +08:00] [mmseqs] [-

100%|██████████| 1/1 [00:00<00:00,  7.20it/s]

✅ ESM2 embeddings with 2 querys generated.


✅ 10 encoded succesfully, 0 failed.


In [6]:
out_embs.keys(), invalid_inputs

(dict_keys(['ETV7', 'IFNGR2', 'IFNGR1', 'ATF2', 'CAV1', 'MARCH8', 'IRF1', 'STAT2', 'IRF7', 'PDL1']),
 [])

In [7]:
import numpy as np

combined_embs = np.concatenate([emb.reshape(1, -1) for emb in out_embs.values()], axis=0)
combined_embs.shape

(10, 256)

## Save embedddings

In [8]:
import pickle

with open('../demo_data/embeddings_output.pkl', 'wb') as f:
    pickle.dump(out_embs, f) 
